In [50]:
!pip install scikit-learn


import pickle
import json
import matplotlib.pyplot as plt
from sklearn.manifold import MDS

import plotly.graph_objects as go

!pip install ipywidgets --upgrade


import os
import ipywidgets as widgets
from IPython.display import display

In [51]:
import os

# Define la ruta base donde se encuentran tus archivos de experimento
base_path = '/content/drive/MyDrive/2023/Trabajo de Grado/DATA'

# Verifica si la ruta base existe
if not os.path.exists(base_path):
    print(f"La ruta especificada no existe: {base_path}")
else:
    print(f"La ruta especificada existe: {base_path}")

    # Lista todas las carpetas y archivos en la ruta base
    all_folders_and_files = os.listdir(base_path)

    # Filtra para obtener solo carpetas
    folders = [item for item in all_folders_and_files if os.path.isdir(os.path.join(base_path, item))]
    print("Carpetas encontradas:", folders)

    # Si hay carpetas, selecciona la primera y lista los archivos .pkl dentro de ella
    if folders:
        first_folder_path = os.path.join(base_path, folders[0])
        pkl_files = [file for file in os.listdir(first_folder_path) if file.endswith('.pkl')]
        print(f"Archivos .pkl en la primera carpeta '{folders[0]}':", pkl_files)
    else:
        print("No se encontraron carpetas dentro de la ruta base.")


La ruta especificada existe: /content/drive/MyDrive/2023/Trabajo de Grado/DATA
Carpetas encontradas: ['Experimento PC set - Escala Mayor - Octava 5 - Intervalos 2']
Archivos .pkl en la primera carpeta 'Experimento PC set - Escala Mayor - Octava 5 - Intervalos 2': ['experiment_data.pkl']


In [52]:
import os
import pickle
import ipywidgets as widgets
from IPython.display import display, clear_output

# Definimos la ruta base
base_path = '/content/drive/MyDrive/2023/Trabajo de Grado/DATA'
global_loaded_data = {}
def load_pkl_files(base_path):
    if not os.path.exists(base_path):
        print(f"La ruta especificada no existe: {base_path}")
        return

    # Obtenemos las carpetas dentro de la ruta base
    folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    folder_selector = widgets.Dropdown(
        options=folders,
        description='Carpeta:',
        disabled=False,
    )

    # Dropdown para los archivos .pkl, inicialmente vacío
    file_selector = widgets.Dropdown(
        options=[],
        description='Archivo:',
        disabled=True,
    )

    # Botón para cargar el archivo seleccionado
    load_button = widgets.Button(description="Cargar datos")

    # Botón para refrescar la lista de archivos después de seleccionar una carpeta
    refresh_button = widgets.Button(description="Refrescar archivos")

    # Display widgets
    display(folder_selector, refresh_button, file_selector, load_button)

    # Función para actualizar la lista de archivos .pkl en la carpeta seleccionada
    def on_refresh_button_clicked(b):
        folder_path = os.path.join(base_path, folder_selector.value)
        if os.path.exists(folder_path):
            pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
            file_selector.options = pkl_files
            file_selector.disabled = False if pkl_files else True
        else:
            print(f"La carpeta seleccionada no existe: {folder_path}")

    refresh_button.on_click(on_refresh_button_clicked)

    # Función para cargar los datos del archivo seleccionado
    def on_load_button_clicked(b):
        if folder_selector.value is None or file_selector.value is None:
            print("Por favor, seleccione una carpeta y un archivo.")
            return experiment_data

        file_path = os.path.join(base_path, folder_selector.value, file_selector.value)
        try:
            with open(file_path, 'rb') as f:
                experiment_data = pickle.load(f)
            global global_loaded_data
            global_loaded_data['experiment_data'] = experiment_data
            print(f"Datos cargados desde {file_path}")
            #load_pkl_files.experiment_data = experiment_data
        except Exception as e:
            print(f"Error al cargar el archivo: {e}")

    load_button.on_click(on_load_button_clicked)


# Llamamos a la función para cargar archivos .pkl
load_pkl_files(base_path)


Dropdown(description='Carpeta:', options=('Experimento PC set - Escala Mayor - Octava 5 - Intervalos 2',), val…

Button(description='Refrescar archivos', style=ButtonStyle())

Dropdown(description='Archivo:', disabled=True, options=(), value=None)

Button(description='Cargar datos', style=ButtonStyle())

In [53]:
# Acceso a los datos cargados
if 'experiment_data' in global_loaded_data:
    experiment_data = global_loaded_data['experiment_data']
    chords = experiment_data['chords']
    # Aquí puedes continuar usando los datos de chords como sea necesario
else:
    print("No se han cargado datos de experimentos aún.")


No se han cargado datos de experimentos aún.


In [54]:
#un Archivo temporal para pruebas
tmp=chords[0]


print(tmp)

{'octave': 5, 'root': 0, 'intervals': [4, 3]}


In [55]:
import numpy as np  # Asegúrate de tener numpy importado para usar np.zeros y np.cumsum

def intervals_extractor(chord):
    # La función intervals_extractor recibe un acorde y extrae los intervalos entre sus notas.

    # Calcula el número total de notas en el acorde.
    # Se suma 1 a la longitud de los intervalos para incluir la nota raíz en el conteo.
    n = len(chord['intervals']) + 1

    # Crea una matriz de ceros con tamaño (n-1) x (n-1).
    # Esta matriz será una matriz triangular superior que representará los intervalos entre las notas.
    chormatrix = np.zeros((n-1, n-1))

    # Calcula las posiciones absolutas de las notas en una escala.
    # Comienza con la nota raíz (posición 0) y luego suma cada intervalo de forma acumulativa.
    note_positions = [0] + list(np.cumsum(chord['intervals']))

    # Llena la matriz con las distancias entre cada par de notas.
    # Recorre solo la parte necesaria de la matriz para mejorar la eficiencia.
    for i in range(n-1):
        for j in range(i+1, n):
            # La distancia entre dos notas se calcula como la diferencia absoluta de sus posiciones.
            dist = abs(note_positions[i] - note_positions[j])

            # Asigna esta distancia a la matriz en la posición correspondiente.
            chormatrix[i, j-1] = dist

    # Devuelve la matriz de intervalos completada.
    return chormatrix


In [64]:

intervals_extractor(tmp)

chord_vectors = []  # Lista para almacenar los vectores generados

In [65]:
for chord in chords:
    vector = intervals_extractor(chord)
    chord_vectors.append(vector)

# Imprime los vectores generados
for vector in chord_vectors:
    print(vector)

[[4. 7.]
 [0. 3.]]
[[3. 7.]
 [0. 4.]]
[[3. 7.]
 [0. 4.]]
[[4. 7.]
 [0. 3.]]
[[4. 7.]
 [0. 3.]]
[[3. 7.]
 [0. 4.]]
[[3. 6.]
 [0. 3.]]


In [73]:
def pitch_classes_extractor(chord,chord_id, repeat=False):
    """
    Extrae las clases de tono de un acorde.

    Parámetros:
    - chord (dict): Un diccionario con las claves "root" e "intervals", donde "root" es la nota raíz y "intervals" es una lista de intervalos.
    - repeat (bool): Si es True, mantiene las clases de tono repetidas. Si es False, elimina las clases de tono repetidas.

    Retorna:
    - Una lista de clases de tono.

    Nota:
    En la teoría de conjuntos de tono, generalmente se manejan conjuntos sin repeticiones. Sin embargo,
    si se desea conservar las repeticiones para un análisis o aplicación específica, se puede hacerlo con el parámetro `repeat`.
    """
    root = chord["root"]
    chord_pitches = [root]

    for interval in chord["intervals"]:
        root += interval
        root %= 12
        chord_pitches.append(root)

    if not repeat:
        chord_pitches = list(set(chord_pitches))
    feature_vector=chord_pitches
    result = {
        "chord": chord,
        "feature_vector": feature_vector,
        "chord_id": chord_id
    }





    return  result

In [74]:
print(chords[2])

pitch_classes_extractor(tmp,0)

{'octave': 5, 'root': 4, 'intervals': [3, 4]}


{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]},
 'feature_vector': [0, 4, 7],
 'chord_id': 0}

In [80]:
def normal_form(pcs):
    """
    Devuelve la forma normal de un conjunto de clases de tono.

    La idea detrás de la forma normal es tomar un conjunto de clases de tono y transformarlo en una forma estándar,
    es decir, una forma que representa todas las transposiciones e inversiones posibles del conjunto.
    Esta es una técnica introducida por Allen Forte en su teoría de conjuntos de clases de tono.

    Parámetros:
    pcs (list): Lista de clases de tono (enteros del 0 al 11).

    Retorna:
    list: Forma normal del conjunto de clases de tono.
    """

    # Ordena el conjunto y crea una lista para almacenar todas las rotaciones posibles
    pcs = sorted(pcs)
    all_rotations = [pcs]

    # Genera todas las rotaciones del conjunto
    for i in range(1, len(pcs)):
        rotation = [(note + 12 - pcs[i]) % 12 for note in pcs]
        all_rotations.append(sorted(rotation))

    # Elige la rotación que esté "más a la izquierda"
    return min(all_rotations, key=lambda x: tuple([(x[i] - x[i - 1]) % 12 for i in range(1, len(x))]))

In [81]:
def interval_vector(pcs):
    """
    Calcula el vector de intervalos para un conjunto de clases de tono.

    El vector de intervalos es un histograma que cuenta la ocurrencia de cada intervalo posible (del 1 al 6) en un conjunto.
    Es una herramienta útil para clasificar y comparar conjuntos en la teoría de Allen Forte.

    Parámetros:
    pcs (list): Lista de clases de tono (enteros del 0 al 11).

    Retorna:
    list: Vector de intervalos del conjunto de clases de tono.
    """

    intervals = [0] * 6

    # Cuenta cada intervalo en el conjunto
    for i in range(len(pcs)):
        for j in range(i + 1, len(pcs)):
            interval = abs(pcs[j] - pcs[i]) % 12
            if interval > 6:
                interval = 12 - interval
            intervals[interval - 1] += 1

    return intervals

In [82]:
def transpose_to_zero(pc_set):
    """Transpone un conjunto de clases de tono para que el primer elemento sea 0."""
    transposition = pc_set[0]
    return [(pitch - transposition) % 12 for pitch in pc_set]

def prime_form(pcs):
    """
    Calculates the prime form of a pitch class set.

    Parameters:
    - pcs (list): A list containing the pitch classes of the set.

    Returns:
    - A list representing the prime form of the set.

    Context:
    Following Allen Forte's theory of pitch-class set, the prime form is the most compact representation of a set.
    It's obtained by comparing the normal form of the original set and its inversion, and then selecting the most "left-ward" form.
    """
    inverted_pcs = [(12 - pitch) % 12 for pitch in pcs]
    inverted_normal_form = normal_form(inverted_pcs)

    if tuple(inverted_normal_form) < tuple(pcs):
        return inverted_normal_form

    return pcs

#Representing Harmony in Computational Music Cognition

Representaciones Propuestas:
Categorías de Representación:

##Acústicas:
 Caracterizan el sonido musical de manera que no depende de un oyente.
##Sensoriales:
Reflejan las imágenes perceptuales de los oyentes de estos sonidos.
##Simbólicas:
 Descripciones concisas y legibles por humanos de los acordes musicales que idealmente tienen una base cognitiva sólida.
Definiciones de "Armonía" y "Acorde":

In [91]:
print(tmp)
pitch_classes_extractor(tmp,0)

{'octave': 5, 'root': 0, 'intervals': [4, 3]}


{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]},
 'feature_vector': [0, 4, 7],
 'chord_id': 0}

In [96]:
def rahn_normal_order(chord, chord_id):
    # Obtenemos las clases de tono desde el diccionario devuelto por pitch_classes_extractor
    pitch_class_set = pitch_classes_extractor(chord, chord_id)["feature_vector"]

    # Convertir el conjunto a un conjunto único para eliminar duplicados y luego a una lista para poder ordenarlo
    pitch_class_set = sorted(list(set(pitch_class_set)))

    # Enumerar todos los posibles ciclos del conjunto de clases de tono
    cycles = [pitch_class_set[i:] + pitch_class_set[:i] for i in range(len(pitch_class_set))]

    # Función para calcular la compactness de un ciclo
    def compactness(cycle):
        # Diferencia entre el último y el primer elemento, módulo 12
        return (cycle[-1] - cycle[0]) % 12

    # Función para aplicar criterios secundarios en caso de empates en compactness
    def secondary_criteria(cycle):
        # Itera desde el segundo hasta el último elemento para desempatar
        for i in range(2, len(cycle) + 1):
            yield (cycle[-i] - cycle[0]) % 12

    # Seleccionar el ciclo más compacto, utilizando primero compactness y luego criterios secundarios
    most_compact_cycle = min(cycles, key=lambda cycle: (compactness(cycle), *secondary_criteria(cycle)))

    # Preparar y devolver el resultado
    result = {
        "chord": chord,
        "feature_vector": most_compact_cycle,
        "chord_id": chord_id
    }
    return result




In [97]:
# Uso de la función
rahn_normal_order_result = rahn_normal_order(tmp, 0)
print(rahn_normal_order_result)

{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]}, 'feature_vector': [0, 4, 7], 'chord_id': 0}


In [98]:
extracted_data = pitch_classes_extractor(tmp,0)
pitch_classes = extracted_data["feature_vector"]

# Ahora usa 'pitch_classes' en otras funciones
nf = normal_form(pitch_classes)
iv = interval_vector(pitch_classes)
pf = prime_form(nf)
rh = rahn_normal_order(tmp,0)
print(nf, iv, pf,rh)

[0, 3, 8] [0, 0, 1, 1, 1, 0] [0, 3, 7] {'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]}, 'feature_vector': [0, 4, 7], 'chord_id': 0}


In [99]:
rahn_normal_order(tmp,0)

{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]},
 'feature_vector': [0, 4, 7],
 'chord_id': 0}

In [102]:
def binary_pitch_class_set(chord, chord_id):
    # Obtenemos las clases de tono desde el diccionario devuelto por pitch_classes_extractor
    pitch_class_set = pitch_classes_extractor(chord, chord_id)["feature_vector"]

    # Representar como vector binario: 1 si la clase de tono está presente, 0 si no
    binary_vector = [1 if i in pitch_class_set else 0 for i in range(12)]

    # Preparar y devolver el resultado
    result = {
        "chord": chord,
        "feature_vector": binary_vector,
        "chord_id": chord_id
    }
    return result




In [103]:
resultado=binary_pitch_class_set(tmp,0)
#resultado["feature_vector"]
print(resultado)

{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]}, 'feature_vector': [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 'chord_id': 0}


In [117]:
def encode_pitch_class_set(chord, chord_id):
    # Obtenemos las clases de tono desde el diccionario devuelto por pitch_classes_extractor
    pitch_class_set = pitch_classes_extractor(chord, chord_id)["feature_vector"]

    # Paso 1: Representar como cadena binaria
    binary_vector = ''.join(['1' if i in pitch_class_set else '0' for i in range(12)])

    # Paso 2 y 3: Convertir la cadena binaria a un número entero (base-10)
    feature_vector = int(binary_vector, 2)

    # Preparar y devolver el resultado
    result = {
        "chord": chord,
        "feature_vector": feature_vector,
        "chord_id": chord_id
    }
    return result


In [120]:
encode_pitch_class_set(tmp,0)

{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]},
 'feature_vector': 2192,
 'chord_id': 0}

In [131]:
def pitch_class_set_type(chord, chord_id):
    normal_order = rahn_normal_order(chord, chord_id)["feature_vector"]
    transposition = (0 - normal_order[0]) % 12
    transposed_set = {(pitch + transposition) % 12 for pitch in normal_order}
    return transposed_set



def encode_pitch_class_set_type():
    types = []

    for i in range(1, 4096):  # 2^12 = 4096, pero 0 no es un conjunto válido
        binary_representation = bin(i)[2:].zfill(12)
        pitch_class_set = {index for index, value in enumerate(binary_representation) if value == '1'}

        # Construir un diccionario de acorde para cada conjunto
        chord = {"root": 0, "intervals": list(pitch_class_set)}
        current_type = pitch_class_set_type(chord, None)

        if current_type not in types:
            types.append(current_type)

    return types




# Probar la codificación con los primeros tipos
pitch_class_set_types = encode_pitch_class_set_type()
first_pitch_class_set_types = pitch_class_set_types[:4]
first_pitch_class_set_types


def encode_pitch_class_chord_type(pitch_class_chord):
    pitch_class_chord = pitch_class_chord - {0}
    binary_vector = ''.join(['1' if i in pitch_class_chord else '0' for i in range(1, 12)])
    return int(binary_vector, 2) + 1



# Probar la codificación con los ejemplos proporcionados
examples_pitch_class_chord_type = [{0}, {0, 1}, {0, 4, 7}]
encoded_chord_types = [encode_pitch_class_chord_type(example) for example in examples_pitch_class_chord_type]
encoded_chord_types

[1, 1025, 145]

Hasta ahora se han extraido dos carateritiscas para cada acorde, su PC set y sus intervalos  desde aui deberiamos unificar esas caractaertisitcas para probar disitions vectores de carcterisiticas.


In [137]:
def generate_interval_histogram(chord):
    # Extraemos la matriz de intervalos
    intervals_matrix = intervals_extractor(chord)

    # Aplanamos la matriz de intervalos y la convertimos a una lista
    intervals_list = intervals_matrix.flatten().tolist()

    # Creamos un histograma con 11dims para los intervalos
    interval_histogram = [0] * 11
    for interval in intervals_list:
        if interval != 0:  # Excluimos la nota raíz
            interval_histogram[int(interval) - 1] += 1

    return interval_histogram








def Experiment_pcset(chord, chord_id):


    # Extraemos el conjunto PC y el histograma de intervalos
    pc_set = pitch_classes_extractor(chord,chord_id)
    interval_histogram = generate_interval_histogram(chord)

    # Combinamos el conjunto PC y el histograma de intervalos en un solo vector de características
    #feature_vector = pc_set + interval_histogram
    feature_vector = pc_set

    # Agregamos el chord_id al resultado
    result = {
        "chord": chord,
        "feature_vector": feature_vector,
        "chord_id": chord_id
    }

    return result







In [144]:
print(tmp)
histogram = generate_interval_histogram(tmp)

print(histogram)
print(intervals_extractor(tmp))
print(pitch_classes_extractor(tmp,0))
generate_interval_histogram(tmp)
#feature_vector = Experiment_pcset(tmp,0)

#print(feature_vector)

{'octave': 5, 'root': 0, 'intervals': [4, 3]}
[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0]
[[4. 7.]
 [0. 3.]]
{'chord': {'octave': 5, 'root': 0, 'intervals': [4, 3]}, 'feature_vector': [0, 4, 7], 'chord_id': 0}


[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0]

In [145]:
def process(chords, experiment_data, func=rahn_normal_order):
    results = {}
    for i, chord in enumerate(chords):
        chord_id = f"chord_{i}"  # Generar un identificador único para el acorde
        result = func(chord, chord_id)  # Llamamos a func en lugar de Experiment_pcset directamente
        #print(result)
        result['chord_id'] = chord_id  # Agregar el identificador único al resultado
        results[chord_id] = result

    # Actualizar el diccionario experiment_data con los resultados
    experiment_data['results'] = results

    # Definir la ruta del directorio y el archivo para guardar los datos
    directory_path = os.path.join('/content/drive/MyDrive/2023/Trabajo de Grado/DATA/Experimento PC set - Escala Mayor - Octava 5 - Intervalos 2')
    experiment_data_file = os.path.join(directory_path, 'experiment_data_'+func.__name__+'.pkl')
    results_file = os.path.join(directory_path, 'results_'+func.__name__+'.pkl')

    # Guardar el diccionario completo en un archivo pickle
    with open(experiment_data_file, 'wb') as f:
        pickle.dump(experiment_data, f)

    # Guardar solo los resultados en un archivo pickle
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

    return experiment_data


process(chords, experiment_data)

{'experiment_name': 'Experimento PC set - Escala Mayor',
 'experiment_params': {'scale': {'name': 'Escala Mayor',
   'root': 0,
   'intervals': [0, 2, 4, 5, 7, 9, 11]},
  'octaves': [5],
  'sizes': [2],
  'intervals': [2]},
 'chords': [{'octave': 5, 'root': 0, 'intervals': [4, 3]},
  {'octave': 5, 'root': 2, 'intervals': [3, 4]},
  {'octave': 5, 'root': 4, 'intervals': [3, 4]},
  {'octave': 5, 'root': 5, 'intervals': [4, 3]},
  {'octave': 5, 'root': 7, 'intervals': [4, 3]},
  {'octave': 5, 'root': 9, 'intervals': [3, 4]},
  {'octave': 5, 'root': 11, 'intervals': [3, 3]}],
 'results': {'chord_0': {'chord': {'octave': 5,
    'root': 0,
    'intervals': [4, 3]},
   'feature_vector': [0, 4, 7],
   'chord_id': 'chord_0'},
  'chord_1': {'chord': {'octave': 5, 'root': 2, 'intervals': [3, 4]},
   'feature_vector': [2, 5, 9],
   'chord_id': 'chord_1'},
  'chord_2': {'chord': {'octave': 5, 'root': 4, 'intervals': [3, 4]},
   'feature_vector': [4, 7, 11],
   'chord_id': 'chord_2'},
  'chord_3': {